### Import packages (cv2 is necessary)

In [2]:
import cv2
import numpy as np
import os
import imageio

## load frames (for large videos only each 5th frame) in pictures

In [5]:
video = imageio.get_reader("checkerboard_bottom_nocrop.mp4")
pictures = [np.asarray(video.get_data(i)) for i in range(video.count_frames()) if i%5 == 0]

In [4]:
video.count_frames()

5270

In [6]:
len(pictures)

1054

## Define Checkerboard and run calibration

In [7]:
CHECKERBOARD = (5,5)

#https://medium.com/@kennethjiang/calibrate-fisheye-lens-using-opencv-333b05afa0b0
subpix_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC + cv2.fisheye.CALIB_CHECK_COND + cv2.fisheye.CALIB_FIX_SKEW
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

### read images and for each image:
n = 0
for img in pictures:
    img_shape = img.shape[:2]

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
    if n%100 == 0:
        print(n)
    n+=1
# calculate K & D
N_imm = len(objpoints)
print("\nnumber of calibration images", N_imm)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_imm)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_imm)]
retval, K, D, rvecs, tvecs = cv2.fisheye.calibrate(
    objpoints,
    imgpoints,
    gray.shape[::-1],
    K,
    D,
    rvecs,
    tvecs,
    calibration_flags,
    (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6))


0
100
200
300
400
500
600
700
800
900
1000
588


## After the calibration is done you can print the camera intrinsics (K, D)

In [8]:
print(K, D)

[[434.84991261   0.         613.3162952 ]
 [  0.         426.34362803 454.6980908 ]
 [  0.           0.           1.        ]] [[-0.05097968]
 [ 0.07257948]
 [-0.08195595]
 [ 0.03209632]]


### as well as the extrinsics

In [2]:
print(rvecs)
print(tvecs)


NameError: name 'rvecs' is not defined

## and the root mean square (RMS) re-projection error
### -usually it should be between 0.1 and 1.0 pixels in a good calibration.

#### The calculation is done by projecting the 3D chessboard points (objectPoints) into the image plane using the final set of calibration parameters (cameraMatrix, distCoeffs, rvecs and tvecs) and comparing the known position of the corners (imagePoints).

#### -An RMS error of 1.0 means that, on average, each of these projected points is 1.0 px away from its actual position. The error can be considered as a distance.

##### https://stackoverflow.com/questions/29628445/meaning-of-the-retval-return-value-in-cv2-calibratecamera

In [ ]:
print(retval)